In [0]:
# Import Relevant Libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from google.colab import files
import io

import folium


from sklearn.cluster import KMeans

from google.colab import files

In [2]:
CLIENT_ID = 'LCI1HINAS0GITQJRCF2OUU0NQDQFXEYUWQYWIDXJS1H3CCLX' # your Foursquare ID
CLIENT_SECRET = 'J4DNTHVLTC1H3FZ31MSH3IEBFSCDUEYSGOSQSMVDLHOADJZ0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LCI1HINAS0GITQJRCF2OUU0NQDQFXEYUWQYWIDXJS1H3CCLX
CLIENT_SECRET:J4DNTHVLTC1H3FZ31MSH3IEBFSCDUEYSGOSQSMVDLHOADJZ0


In [0]:
radius=1000
LIMIT = 100
latitude = 18.939277
longitude = -99.227159


url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [0]:
#results = requests.get(url).json()
#results

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
with open('worldcitiespop.csv', 'r') as f:
  gc = pd.read_csv(f)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
Morelos_mask = (gc.Country == 'mx')# & (gc.Region == 17)
TX_mask = (gc.Country == 'us') & (gc.Region.isin(['TX','CA','NM','AZ']))
mask = Morelos_mask | TX_mask

In [0]:
Morelos_cities = gc[Morelos_mask].sort_values(by='Population', ascending = False).head(50)
TX_cities = gc[TX_mask].sort_values(by='Population', ascending = False).head(50)

In [0]:
data = pd.concat([TX_cities,Morelos_cities]).sort_values(by='Population', ascending = False)

data['Name'] = data.AccentCity + ", " + data.Region.astype(str)

In [24]:
data.head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude,Name
1892345,mx,mexico,Mexico,9,8720916.0,19.434167,-99.138611,"Mexico, 9"
2920652,us,los angeles,Los Angeles,CA,3877129.0,34.052222,-118.242778,"Los Angeles, CA"
3027895,us,houston,Houston,TX,2027712.0,29.763056,-95.363056,"Houston, TX"
1869323,mx,ecatepec,Ecatepec,15,1806297.0,19.601111,-99.052500,"Ecatepec, 15"
1878045,mx,guadalajara,Guadalajara,14,1640649.0,20.666667,-103.333333,"Guadalajara, 14"


In [25]:
# create map of North America using latitude and longitude values

latitude = 33
longitude = -100

map_TXMX = folium.Map(location=[latitude, longitude], zoom_start=5)



# add markers to map
for lat, lng, Name in zip(data['Latitude'], data['Longitude'], data['Name']):
    label = '{}'.format(Name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TXMX)  
    
map_TXMX

In [0]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
#Here we'll create a data frame of venues in each neighborhood
venues = getNearbyVenues(names=data['Name'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

Mexico, 9
Los Angeles, CA
Houston, TX
Ecatepec, 15
Guadalajara, 14
Phoenix, AZ
Juarez, 15
Puebla, 21
Tijuana, 2
San Diego, CA
San Antonio, TX
Nezahualcóyotl, 15
Dallas, TX
Monterrey, 19
León, 11
Zapopan, 14
San Jose, CA
Naucalpan, 15
San Francisco, CA
Guadalupe, 19
Mérida, 31
Tlalnepantla, 15
Chihuahua, 6
Austin, TX
San Luis Potosí, 24
Aguascalientes, 1
Acapulco, 12
Saltillo, 7
Fort Worth, TX
Querétaro, 22
El Paso, TX
Mexicali, 2
Hermosillo, 26
Morelia, 16
Chimalhuacán, 15
Culiacán, 25
Veracruz, 30
Cancún, 23
Torreón, 7
López Mateos, 15
Tucson, AZ
San Nicolás de los Garza, 19
Toluca, 15
Reynosa, 28
Tlaquepaque, 14
Albuquerque, NM
Long Beach, CA
Tuxtla Gutiérrez, 5
Cuautitlán Izcalli, 15
Sacramento, CA
Fresno, CA
Durango, 10
Mesa, AZ
Matamoros, 28
Xalapa, 30
Tonalá, 14
Oakland, CA
Arlington, TX
Xico, 15
Villahermosa, 27
Mazatlán, 25
Apodaca, 19
Ixtapaluca, 15
Nuevo Laredo, 28
Cuernavaca, 17
Santa Ana, CA
Irapuato, 11
Anaheim, CA
Pachuca, 13
Coacalco, 15
Tampico, 28
Riverside, CA
Bakersf

In [28]:
print(venues.shape)
venues.head()

(9918, 7)


,Name,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Mexico, 9",19.434167,-99.138611,Museo Nacional de Arte (MUNAL),19.436018,-99.139603,Art Museum
1,"Mexico, 9",19.434167,-99.138611,Hotel Historico Central,19.433153,-99.138376,Hotel
2,"Mexico, 9",19.434167,-99.138611,Palacio de Cultura Banamex - Antiguo Palacio d...,19.433681,-99.139018,Art Museum
3,"Mexico, 9",19.434167,-99.138611,Quinta Casa de Correos (Palacio Postal),19.435820,-99.140376,Post Office
4,"Mexico, 9",19.434167,-99.138611,Palacio de Bellas Artes,19.434672,-99.141289,Opera House


In [29]:
#Lets see how many venues we have in each city
venues.groupby('Name').count().head(50)

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Name,,,,,,
"Acapulco, 12",100,100,100,100,100,100
"Aguascalientes, 1",100,100,100,100,100,100
"Albuquerque, NM",100,100,100,100,100,100
"Amarillo, TX",100,100,100,100,100,100
"Anaheim, CA",100,100,100,100,100,100
"Apodaca, 19",100,100,100,100,100,100
"Arlington, TX",100,100,100,100,100,100
"Austin, TX",100,100,100,100,100,100
"Bakersfield, CA",100,100,100,100,100,100


In [30]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 389 uniques categories.


In [31]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['City Name'] = venues['Name'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,City Name,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Yucatecan Restaurant,Zoo,Zoo Exhibit
0,"Mexico, 9",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Mexico, 9",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Mexico, 9",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Mexico, 9",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Mexico, 9",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
grouped = onehot.groupby("City Name").mean().reset_index()
grouped

,City Name,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Yucatecan Restaurant,Zoo,Zoo Exhibit
0,"Acapulco, 12",0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00
1,"Aguascalientes, 1",0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.010000,0.00,0.00,0.0,0.00,0.00
2,"Albuquerque, NM",0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.050000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.01,0.00
3,"Amarillo, TX",0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.060000,0.000000,...,0.00,0.01,0.00,0.00,0.010000,0.00,0.00,0.0,0.01,0.00
4,"Anaheim, CA",0.0,0.01,0.00,0.00,0.00,0.0,0.0,0.020000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00
5,"Apodaca, 19",0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00
6,"Arlington, TX",0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.030000,0.000000,...,0.00,0.00,0.00,0.00,0.010000,0.00,0.00,0.0,0.00,0.00
7,"Austin, TX",0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.010000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.00,0.02,0.0,0.00,0.00
8,"Bakersfield, CA",0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.010000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00
9,"Brownsville, TX",0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.010000,0.000000,...,0.00,0.00,0.00,0.00,0.020000,0.00,0.00,0.0,0.01,0.00


In [33]:
num_top_venues = 3

for cty in grouped['City Name']:
    print("----"+cty+"----")
    temp = grouped[grouped['City Name'] == cty].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acapulco, 12----
                venue  freq
0  Seafood Restaurant  0.12
1  Mexican Restaurant  0.11
2               Beach  0.09


----Aguascalientes, 1----
                venue  freq
0  Mexican Restaurant  0.15
1          Taco Place  0.12
2                 Bar  0.08


----Albuquerque, NM----
                venue  freq
0  Mexican Restaurant  0.13
1             Brewery  0.07
2         Pizza Place  0.06


----Amarillo, TX----
                  venue  freq
0    Mexican Restaurant  0.08
1  Fast Food Restaurant  0.07
2   American Restaurant  0.06


----Anaheim, CA----
                          venue  freq
0  Theme Park Ride / Attraction  0.30
1                    Theme Park  0.11
2                  Burger Joint  0.04


----Apodaca, 19----
                venue  freq
0  Mexican Restaurant  0.13
1          Taco Place  0.10
2      Ice Cream Shop  0.05


----Arlington, TX----
                 venue  freq
0   Mexican Restaurant  0.05
1          Coffee Shop  0.05
2  Fried Chicken Joint  0.0

In [0]:
def return_most_common_venues(df, num):
  df.drop("City Name", inplace = True)
  cols = pd.to_numeric(df).nlargest(num).index.values.tolist()
  return cols

In [35]:
testdf=grouped.iloc[0, :].drop("City Name")
pd.to_numeric(testdf).nlargest(10).index.values.tolist()

['Seafood Restaurant',
 'Mexican Restaurant',
 'Beach',
 'Taco Place',
 'Burger Joint',
 'Coffee Shop',
 'Food Truck',
 'Ice Cream Shop',
 'Restaurant',
 'Hotel']

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['Name'] = grouped['City Name']

for ind in np.arange(grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(obj)


,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Acapulco, 12",Seafood Restaurant,Mexican Restaurant,Beach,Taco Place,Burger Joint,Coffee Shop,Food Truck,Ice Cream Shop,Restaurant,Hotel
1,"Aguascalientes, 1",Mexican Restaurant,Taco Place,Bar,Ice Cream Shop,Bakery,Plaza,Art Museum,Food Truck,Garden,Hotel
2,"Albuquerque, NM",Mexican Restaurant,Brewery,Pizza Place,American Restaurant,Sandwich Place,Hotel,Café,Coffee Shop,Theater,Bar
3,"Amarillo, TX",Mexican Restaurant,Fast Food Restaurant,American Restaurant,Sandwich Place,Burger Joint,Coffee Shop,Restaurant,Discount Store,BBQ Joint,Grocery Store
4,"Anaheim, CA",Theme Park Ride / Attraction,Theme Park,Burger Joint,Gift Shop,Hotel,Ice Cream Shop,American Restaurant,Art Gallery,Brewery,Cajun / Creole Restaurant


In [0]:
## Now let's cluster the cities

In [0]:
# set number of clusters
kclusters = 30

grouped_clustering = grouped.drop('City Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:50] 

In [0]:
# add clustering labels
city_venues_sorted['Cluster Labels']= kmeans.labels_

In [0]:
# Merge lat long into city venues

merged = data.join(city_venues_sorted.set_index('Name'), on='Name')

In [0]:
#merged.head()

In [0]:
merged = merged.dropna()

In [43]:
## Lets map out the clusters. The colors show which luster the neighborhood belongs to

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5.2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Name'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
clstrs = merged.groupby("Cluster Labels").Country.agg([lambda x: sum(x == 'us')/x.count(),'count']).sort_values(by = '<lambda>')
clstrs.columns = ['%us','count']
clstrs[(clstrs['%us'] != 1.0) &(clstrs['%us'] != 0.0)]
#These are the clusters with US and Mexican cities

,%us,count
Cluster Labels,,
11,0.125000,8
3,0.333333,3
7,0.500000,2
22,0.500000,2


In [49]:
City = 'Austin'

Cluster = 11#merged[merged.AccentCity == City]['Cluster Labels'].values[0]

merged.loc[merged['Cluster Labels'] == Cluster, merged.columns[[0] + list(range(7, merged.shape[1]))]].sort_values(by = 'Country')

,Country,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
1918739,mx,"Zapopan, 14",Italian Restaurant,Mexican Restaurant,Ice Cream Shop,Coffee Shop,Restaurant,Seafood Restaurant,Shopping Mall,Bakery,Gym,Hotel,11
1892331,mx,"Mexicali, 2",Mexican Restaurant,Taco Place,Chinese Restaurant,Coffee Shop,Bar,Café,Salad Place,Art Gallery,BBQ Joint,Bakery,11
1878973,mx,"Hermosillo, 26",Mexican Restaurant,Taco Place,Seafood Restaurant,Coffee Shop,Ice Cream Shop,Bar,Burger Joint,Café,Cupcake Shop,Fast Food Restaurant,11
1893339,mx,"Morelia, 16",Mexican Restaurant,Taco Place,Fast Food Restaurant,Gym / Fitness Center,Movie Theater,Restaurant,Steakhouse,Argentinian Restaurant,Café,Clothing Store,11
1916933,mx,"Veracruz, 30",Mexican Restaurant,Taco Place,Café,Restaurant,Seafood Restaurant,Hotel,Ice Cream Shop,Bar,Historic Site,Italian Restaurant,11
1888771,mx,"López Mateos, 15",Taco Place,Mexican Restaurant,Ice Cream Shop,Dessert Shop,Bakery,Golf Course,Multiplex,Restaurant,Seafood Restaurant,Bar,11
1917271,mx,"Villahermosa, 27",Mexican Restaurant,Ice Cream Shop,Seafood Restaurant,Taco Place,Burger Joint,Coffee Shop,Gym,Restaurant,Sandwich Place,Bakery,11
3026640,us,"Corpus Christi, TX",Mexican Restaurant,Italian Restaurant,Convenience Store,Seafood Restaurant,Bar,Beach,Hotel,Art Gallery,BBQ Joint,Burger Joint,11
